# 使用 LoRA 对模型进行微调

## LoRA 原理

**LoRA（Low-Rank Adaptation，低秩自适应）** 是一种高效微调大语言模型（LLM）的技术，其核心思想是通过低秩分解，在原始模型参数旁添加少量可训练的“低秩矩阵”来模拟参数更新，而非直接调整庞大的原始参数。这种方法大幅减少了训练参数量（通常降低万倍以上），显著节省计算资源和存储开销，同时保持与全参数微调相近的效果，且推理时可通过合并矩阵避免额外延迟。LoRA特别适合资源有限或需要快速适配多任务的大模型场景。其原理图如下所示：

![LoRA Struct](img/LoRA-Struct.svg)

（图中左侧表示全参数微调，右侧表示 LoRA 微调，蓝色部分表示需要训练的权重，灰色部分表示冻结的权重不参与训练）

经过 LoRA 微调后的模型结果会产生一个额外的 LoRA 权重对应于图中的虚线框部分，下次调用模型时将 LoRA 权重加载到基础模型上即可。

## LoRA 训练

在开源模型技术蓬勃发展的当下，以 LoRA 为代表的参数高效微调方法已成为大模型适配下游任务的重要范式。尽管当前主流大语言模型的架构参数大多已开源，**但工程实践中开发者更关注微调策略的实施而非底层网络结构的实现细节**。为此，本文聚焦大语言模型的微调训练实践，基于Hugging Face生态中的强化学习训练库TRL（Transformer Reinforcement Learning），系统解析监督式微调（SFT）等核心技术的实现路径。

TRL作为🤗 Transformers生态的重要组件，集成了监督式微调（SFT）、近端策略优化（PPO）和直接偏好优化（DPO）等前沿训练范式。该库不仅支持多模态模型的灵活适配，更能通过分布式训练框架实现从单卡到集群的弹性扩展，为不同硬件环境下的模型调优提供统一的技术解决方案。其模块化设计兼顾算法创新与工程落地，开发者既可直接调用预置训练流程，也可基于API灵活构建定制化训练策略。

- TRL 开源：[Github - trl](https://github.com/huggingface/trl)
- TRL 文档：[huggingface - trl](https://hf.co/docs/trl)

### 1. 引入必要的库

In [ ]:
from trl import SFTTrainer
from datasets import load_dataset

### 2. 准备数据

In [ ]:
dataset = load_dataset("trl-lib/Capybara", split="train")

### 3. 进行训练

In [ ]:
trainer = SFTTrainer(
    model="Qwen/Qwen2.5-0.5B",
    train_dataset=dataset,
)
trainer.train()